### Introduction
The hedge fund of U.S. company invests long-term investment instead of short-term investment to have highest possible percentage of algorithmic trading to eliminate their redundant cost. 

### Problem statement
In this scenario, the main issue is the volatile stock prices which may results a large degree of variance and the invest into the company riskier. To reduce the risk, the hedge fund's manager asked to develop machine learning model to identify companies that would make good investment. Here, existing data pipeline need to be used with the machine learnng model to predict the future performance of the U.S. stocks. The data will be collected from the top 500 U.S. companies, taking around 150 different financial indicators. In this case, a master data will be counting and analysing with key financial features such as revenue, profit and operating expenses where stock price growth will be the base feature from a business perspective for the machine learning model which can help to predict the good investment in the future.

To continue the algorithmic operation,the master dataset containing financial data has been used with various features. The given feature or targeted feature `stock price growth` will help to build a predictive model by evaluating it's effectiveness and providing the informed decisions as well as will filter the good investment by adding some important features from the share market companies.  

## Data Ingestion and Wrangling

In [1]:
import numpy as np # To enhance the readability of the code and numerical analysis, `numpy` library is imported as `np`.
import pandas as pd # To retrieve the dataset `pandas` library has been imported as `pd`.

In [2]:
NUMBER=540 # Here, defined number or observations are 540. 

### Extracted names of companies and data sources

In [3]:
companies = pd.read_csv('data/companies.csv', header=None, squeeze=True) # Here, `companies` dataframe has been read using 
# `read_csv()` function with pandas library from data folder. This dataframe doesn't have any header and removed the empty space 
# using the `squeeze=True` argument.

In [4]:
data_sources = ['key-metrics', 'income-statement', 'financial-growth', 'ratios'] # Aggregated some financial data using a 
# list `[]` function and making an object named as `data_sources`.

### Joined each company for each method into a single dataframe (concatenate rows)

In [5]:
for data_source in data_sources: # Here, a `for loop` is used to make the `data_source` in `data_sources` by iterating over a sequence such as a list.  
    frames = [] # The `frames` object is made with the empty list.
    for company in companies[:NUMBER]: # Another `for loop` has made the `company` in `companies` with all 540 rows using the list `[]`.
        # The dataframe is appended by reading the data from data folder with `data_source` and `company`.
        frames.append(pd.read_csv('data/' + data_source + '/' + company + '.csv')) 
    # Updated the dataframe using `concat()` function.
    data = pd.concat(frames) 
    data.to_csv('data/' + data_source + '.csv', index=False) # Pandas DataFrame to CSV with no index has shown by using `index=False` param of `to_csv()` method

Joined each data source into a single dataframe (concatenate columns). This will allow us to build our models, and to try conclusions from the base data, as we will have all relevant data in one place, formatted and set up correctly for use and analysis.

In [6]:
# Here, another frame with empty list is created and a `for loop` is used to make the `data_source` in `data_sources` by 
# iterating over a sequence such as a list. The dataframe is appended by reading the data from data folder with `data_source` 
# and `.csv` files.
frames = [] 
for data_source in data_sources:
    # Read data source and append to list
    frames.append(pd.read_csv('data/' + data_source + '.csv'))

In [7]:
# Concatenated columns of dataframes in list
data = pd.concat(frames, axis=1)


In [8]:
# Removed any duplicate columns using the `.loc[]` function with `duplicated()` function.
data = data.loc[:,~data.columns.duplicated()]


In [9]:
# Converted string representation to datetime data type
data['date'] = pd.to_datetime(data['date'])


In [10]:
# Grouped by stock name using `groupby()` function and kept latest record of `date` from the final data.
data = data[data['date']== data.groupby('symbol')['date'].transform('max')]

In [11]:
# Dropped non-numeric rows (datetimes and strings) by applying `drop()` function.
data.drop(['date', 'fillingDate', 'acceptedDate', 'period', 'link', 'finalLink'], axis=1, inplace=True)


In [12]:
# Sorted by market capitalisation using `sort_values()` function where `ascending=False` implies the data in descending order 
# (i.e., from highest to lowest) and `inplace=True` changes the actual list itself while sorting.
data.sort_values('marketCap', ascending=False, inplace=True)


In [13]:
# Updated dataframe using pandas `DataFrame.to_csv()` function which converts DataFrame into CSV data.
data.to_csv('data/data.csv', index=False)

Found differences in stock and company names.

In [14]:
# Here, the `set()` function is used to create a set in python where one parameter will be used such as `drop=True` and
#`stocks` dataframe is generated by reading the `stocks.csv` file from the data folder with two parameter and using `read_csv()` function. 
companies = set(data['symbol'].reset_index(drop=True))
stocks = pd.read_csv('data/stocks.csv', header=None, squeeze=True) 
#stocks = set(stocks.head(NUMBER).reset_index(drop=True))
#stocks.difference(companies.intersection(stocks))

Calculated annual stock price growth which will be the target variable and will help to define a company's good investment according to the above statement. Here, the target variable will be created through feature engineering in the data modelling step.

In [15]:
frames = []
for stock in stocks[:NUMBER]:
    prices = pd.read_csv('data/historical-price/' + stock + '.csv') # Read the `historical-price` data using `read_csv()` function.
    share_growth = (prices.tail(1)['Close'].iloc[0] - prices.head(1)['Close'].iloc[0])/prices.head(1)['Close'].iloc[0]
    frames.append(pd.DataFrame({'symbol': stock.replace('-', ''), 'shareGrowth': share_growth}, index=[stock.replace('-', '')]))
    #frames.append(pd.read_csv('data/historical-price/' + stock + '.csv'))
historical_price = pd.concat(frames).reset_index(drop=True)
historical_price.to_csv('data/historical-price.csv', index=False)

Joined annual stock price growth feature to master dataframe

In [16]:
data = data.set_index('symbol').join(historical_price.set_index('symbol'), how='outer')
data.to_csv('data/data.csv', index=False)

Here, a single dataset has been made where companies names are organised as rows, financial features are set as columns with numeric values.

In [17]:
data

,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,...,priceToBookRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,enterpriseValueMultiple,priceFairValue,shareGrowth
symbol,,,,,,,,,,,,,,,,,,,,,
A,16.235849,3.367925,3.210692,2.720126,4.345912,14.930818,14.943396,14.930818,7.798742,2.551209e+10,...,5.373229,23.820813,29.493746,24.987356,24.987356,23.971578,4.941331,21.556532,5.373229,0.348744
AAP,136.429467,6.841790,12.181676,5.554149,5.883018,49.871159,134.146406,49.871159,11.061870,9.834185e+09,...,2.770910,20.197711,24.880230,11.343964,11.343964,3.391248,1.012893,11.094621,2.770910,-0.082432
AAPL,15.820258,3.308587,4.649230,4.228014,2.190856,3.765477,18.665617,3.765477,6.191751,1.996361e+12,...,30.553901,34.773150,27.211359,24.746031,24.746031,10.919861,7.272322,26.709057,30.553901,1.062603
ABBV,22.416442,5.311321,8.978437,8.606469,26.902965,-5.506739,36.969003,-5.506739,43.638140,1.181362e+11,...,-14.456216,14.988099,9.249624,8.866421,8.866421,3.911085,3.551259,11.547166,-14.456216,0.196094
ABC,932.434060,-16.692812,22.576262,22.754819,22.576262,-5.003211,175.208952,-5.003211,19.550988,1.956504e+10,...,-19.201667,-5.755172,4.221963,4.255355,4.255355,-1.370250,0.103031,-3.939804,-19.201667,0.180224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZBRA,83.069017,10.075707,12.687241,11.557429,0.555646,34.061075,33.598037,34.061075,21.651657,1.329912e+10,...,7.231715,24.446920,21.312699,19.414780,19.414780,3.135201,2.965245,15.943472,7.231715,0.265210
ZEN,7.381299,-1.533850,0.807018,0.382258,1.777399,4.140680,11.823102,4.140680,7.206933,9.622722e+09,...,21.011044,-56.720023,227.595128,107.804327,107.804327,47.859196,11.786543,-99.002389,21.011044,0.415486
ZM,2.448537,0.099509,0.597299,0.446983,1.113394,3.279393,5.072179,3.279393,0.053740,2.670129e+10,...,32.018125,1055.178465,234.908030,175.791293,175.791293,35381.036890,42.882756,467.768420,32.018125,5.201161
